<a href="https://colab.research.google.com/github/kullawattana/thesis_2020_spacy_colab/blob/master/5_nlp_tutorial_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import sys
from spacy.matcher import Matcher

#LOAD MODEL
nlp = spacy.load("en_core_web_sm")

#text = "I can promise it is worth your time."
#text = "We can overtake them."
#text = "I want a pizza and cola."
text = "I want a dish."
doc = nlp(text)

In [ ]:
#STEP 1 ================CHECK GET THE Doc=========================
text = "I want a green apple."
print(w for w in doc[3].lefts)
print(w for w in doc[3].children)
#print(doc[7].tag_)
#print(spacy.explain(doc[7].tag_))

<generator object <genexpr> at 0x7fa8d235c048>
<generator object <genexpr> at 0x7fa8d235c048>


In [ ]:
#STEP 2 ================SHOW DETAILS=========================
#for word in doc:
    #print({word.text:{12}} {word.pos_:{10}} {word.tag_:{8}} {spacy.explain(word.tag_)})

In [ ]:
#STEP 3 ================CONNECT 2 SENTENCE=========================
#connection between two sentences
for i,sent in enumerate(doc.sents):
    print(i)
    if(i==0 and sent[0].pos_ == 'PRON'):
        print(sent[0].text)
        print('The second sentence begins with a pronoun.')

counter = 0
for sent in doc.sents:
    if sent[len(sent)-2].pos_ == 'VERB':
        counter += 1
    print(counter)

0
I
The second sentence begins with a pronoun.
0


In [ ]:
#STEP 4 =============CHUNKS=========================
for chunk in doc.noun_chunks:
    print("chunk :")
    print(chunk)

chunk :
I
chunk :
a dish


In [ ]:
#STEP 5 =============COMPOSE DET with ADJ=======================
for token in doc:
    if token.pos_ == 'NOUN':
        chunk = ''
        for w in token.children:
            if w.pos_ == 'DET' or w.pos_ == 'ADJ':
                chunk = chunk + w.text + ' '
        chunk = chunk + token.text
        print(chunk)

a dish


In [ ]:
#STEP 6 =============PRINT SENTENCE=======================
text = "The Golden Gate Bridge is an iconic landmark in San Francisco."
print([doc[i] for i in range(len(doc))])

[I, want, a, dish, .]


In [ ]:
#STEP 7 =============MERGE WORDING=======================
#MERGE wording Golden, Gate, Bridge => Golden Gate Bridge
span = doc[1:4]
lem_id = doc.vocab.strings[span.text]
print(span.merge(lemma = lem_id))

want a dish


In [ ]:
#STEP 8 =============Numeric, Symbolic, and Punctuation Tags=======================
text = "The firm earned $1.5 million in 2017"
#“Numeric, Symbolic, and Punctuation Tags”
#for token in doc:
    #print({token.text:{12}} {token.pos_:{10}} {token.tag_:{8}} {token.dep_:{8}} {spacy.explain(token.pos_)})

phrase = ''
for token in doc:
    if token.tag_ == 'CD':
        phrase = token.text
        i = token.i+1
        while doc[i].tag_ == 'CD':
            phrase += doc[i].text + ''
            i += 1
        break

phrase = phrase[:-1]
print(phrase)

In [ ]:
#STEP 9 ================“The Use of Pronouns in a Chatbot”===========================
sent = ''
for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and doc[i+1].tag_ == 'MD' and doc[i+2].tag_ == 'VB':
        sent = doc[i+1].text.capitalize() + ' ' + doc[i].text
        sent = sent + ' ' + doc[i+2:].text
        break
    #By now, you should have : 'Can I promise it is worth your time.'
    #Re tokenization

for i,token in enumerate(doc):
    if token.tag_ == 'PRP' and token.text == 'I':
        sent = doc[:i].text + ' you ' + doc[i+1:].text
        break

for i,token in enumerate(doc):
    if token.tag_ == 'PRP$' and token.text == 'your':
        sent = doc[:i].text + ' my ' + doc[i+1:].text
        break

for i,token in enumerate(doc):
    if token.tag_ == 'VB':
        sent = doc[:i].text + ' really ' + doc[i:].text
        break

sent = doc[:len(doc) - 1].text + '?'
print("===============sent==================")
print(sent)

for token in doc:
    if token.dep_ == 'dobj':
        shift = len([w for w in token.children])
        print(shift)
        chunk = doc[i-shift:i+1]
        print(chunk)
        break

===============sent==================
I want a dish?


In [ ]:
#STEP 10 ================#subject + auxiliary + verb” pattern ==========================
#USE DATA text = "We can overtake them."
def dep_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'nsubj' and doc[i+1].dep_ == 'aux' and doc[i+2].dep_ == 'ROOT':
            for tok in doc[i+2].children:
                if tok.dep_ == 'dobj':
                    return True
                return False

if dep_pattern(doc):
    print('Found')
else:
    print('Not found')

Not found


In [ ]:
#STEP 11 ==============“Using spaCy’s Matcher to Find Word Sequence Patterns”===================
text = "We can overtake them."
matcher = Matcher(nlp.vocab)
pattern = [{"DEP":"nsubj"},{"DEP":"aux"},{"DEP":"ROOT"}]
matcher.add("NsubjAuxRoot", None, pattern)
#doc = nlpframework(u"We can overtake them.")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print("Span: ", span.text)
    print("The positions in the doc are: ", start, " - ", end)

In [ ]:
#STEP 12 =============pattern: “personal pronoun + modal auxiliary verb + base form verb + . . . + personal pronoun . . .”.”=================
text = "We can overtake them."
def pos_pattern(doc):
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ != 'PRP':
            return False
        if token.dep_ == 'aux' and token.tag_ != 'MD':
            return False
        if token.dep_ == 'ROOT' and token.tag_ != 'VB':
            return False
        if token.dep_ == 'dobj' and token.tag_ != 'PRP':
            return False
    return True

#Testing code
if dep_pattern(doc) and pos_pattern(doc):
    print('Found')
else:
    print('Not found')

Not found


In [ ]:
#STEP 13==============“Creating Patterns Based on Customized Features”====================
def pron_pattern(doc):
    plural = ['we','us','they','them']
    for token in doc:
        if token.dep_ == 'dobj' and token.tag_ == 'PRP':
            if token.text in plural:
                return 'plural'
            else:
                return 'singular'
        return 'not found'

#การต่่อ String ใช้ (str, str, str)
if dep_pattern(doc) and pos_pattern(doc):
    print('Found:', 'the pronoun in position of direct object is', pron_pattern(doc))
else:
    print('Not found')

Not found


In [ ]:
#STEP 14==============Find Noun by create Pattern List==========================
def find_noun(sents, num):
    if num == 'plural':
        taglist = ['NNS','NNPS']
    if num == 'singular':
        taglist = ['NN','NNP']

    for sent in reversed(sents):
        for token in sent:
            if token.tag_ in taglist:
                return token.text
            return 'Noun not found'

In [ ]:
#STEP 15============gen_utterance====================
def gen_utterance(doc, noun):
    sent = ''
    for i,token in enumerate(doc):
        if token.dep_ == 'dobj' and token.tag_ == 'PRP':
            sent = doc[:i].text + ' ' + noun + ' ' + doc[i+1:len(doc)]
            return sent
        return 'Failed to generate an utterance'

In [ ]:
#STEP 16
doc = nlp(u'The symbols are clearly distinguishable. I can recognize them promptly')
sents = list(doc.sents)
response = ''
noun = ''
for i, sent in enumerate(sents):
    if dep_pattern(sent) and pos_pattern(sent):
        noun = find_noun(sents[:i], pron_pattern(sent))
        if noun != 'Noun not found':
            response = gen_utterance(sents[i], noun)
            break
        print(response)

In [ ]:
#STEP 17============example of premodifiers and postmodifiers===================
doc = nlp(u"Kiwano has jelly-like flesh with a refreshingly fruity taste. This is a nice exotic fruit from Africa. It is definitely worth trying")
fruit_adjectives = []
fruit_origins = []
for token in doc:
    if token.text == 'fruit':
        fruit_adjectives = fruit_adjectives + [modifier.text for modifier in token.lefts if modifier.pos_ == 'ADJ']
        fruit_origins = fruit_origins + [doc[modifier.i + 1].text for modifier in token.rights if modifier.text == 'from' and type != 0]
        print('The list of adjectival modifiers for word fruit:', fruit_adjectives)
        print('The list of GPE names applicable to word fruit as postmodifiers:', fruit_origins)

The list of adjectival modifiers for word fruit: ['nice', 'exotic']
The list of GPE names applicable to word fruit as postmodifiers: ['Africa']


In [ ]:
#STEP 18===================“Extracting Multiple Intents with token.conjuncts”============================
text = nlp(u'I want a pizza and cola.')
#Extract direct object and the conjunct associated with it
for token in doc:
    if token.dep_ == 'dobj':
        dobj = [token.text]
        conj = [t.text for t in token.conjuncts]
        #compose the list of the extracted elements
        dobj_conj = dobj + conj
        print(dobj_conj)

['flesh']


In [ ]:
#STEP 19===================“Using Word Lists to Extract the Intent”====================
def extract_intent(doc):
    dobj = ''
    tverb = ''
    for token in doc:
        if token.dep_ == 'dobj':
            dobj = token
            tverb = token.head
            print("HEAD :", tverb)

    #extract the verb for the intent's definition
    intentVerb = ''
    verbList = ['want', 'like', 'need', 'order']
    print('tverb :', tverb.text)
    if tverb.text in verbList:
        intentVerb = tverb
    else:
        if tverb.head.dep_ == 'ROOT':
            intentVerb = tverb.head

    #extract the object for the intent's definition
    # USE DATA text = "I want a pizza and cola."
    intentObj = ''
    objList = ['pizza', 'cola']
    if dobj.text in objList:
        intentObj = dobj
    else:
        for child in dobj.children:
            if child.dep_ == 'prep':
                intentObj = list(child.children)[0]
                break
            elif child.dep_ == 'compound':
                intentObj = child
                break
    #print the intent expressed in the sample sentence
    print(intentVerb.text + intentObj.text.capitalize())

In [ ]:
#STEP 20===============“Finding the Meanings of Words Using Synonyms and Semantic Similarity”================
# USE DATA text = I want a dish. I'd like to order a pizza. Give me a pie.
for token in doc:
    verb = ''
    dobj = ''

    print("DOC :" , token.text)
    if token.dep_ == 'dobj':
        verb = token.head.text
        dobj = token.text
        print("VERB :", verb, "DOBJ :", dobj)

    #create a list of tuples for possible verb synonyms
    verbList = [('order', 'want', 'give','make'), ('show','find')]
    #find the tuple containing the transitive verb extracted from the sample
    verbSyns = [item for item in verbList if verb in item]
    #create a list of tuples for possible direct object synonyms
    dobjList = [('pizza', 'pie', 'dish'), ('cola', 'soda')]
    #find the tuple containing the direct object extracted from the sample
    dobjSyns = [item for item in dobjList if dobj in item]
    #replace the transitive verb and the direct object with synonyms supported by the application
    #and compose the string that represents the intent
    #print(verbSyns[0][0])
    #print(dobjSyns[0][0].capitalize())
    #intent = verbSyns[0][0] + dobjSyns[0][0].capitalize()
    #print(intent)

DOC : Kiwano
DOC : has
DOC : jelly
DOC : -
DOC : like
DOC : flesh
VERB : has DOBJ : flesh
DOC : with
DOC : a
DOC : refreshingly
DOC : fruity
DOC : taste
DOC : .
DOC : This
DOC : is
DOC : a
DOC : nice
DOC : exotic
DOC : fruit
DOC : from
DOC : Africa
DOC : .
DOC : It
DOC : is
DOC : definitely
DOC : worth
DOC : trying
